### 1. 算法简介


- **简介**：MSPCA可以看作是对经典主成分分析（PCA）、稀疏主成分分析（SPCA）以及最近提出的多线性主成分分析（MPCA）的进一步扩展。

- **关键操作**：将MPCA重写为多线性回归形式，并将其松弛为稀疏回归。

- **算法特点**：与最近提出的MPCA不同，MSPCA继承了SPCA的稀疏性，并迭代地学习一系列稀疏投影，捕获了张量数据大部分的变化。

- **非零元素选取**：稀疏投影中的每个非零元素都是使用弹性网从最重要的变量/因子中选择的。

### 2. 算法流程

#### 2.1. 符号说明

在本文中，如果没有特别说明，斜体的小写字母，如 $\alpha, \beta, k, i, j$ 表示标量，粗体小写字母，如 $\mathbf{u}, \mathbf{v}$ 表示向量，大写字母，如 $U, V, B, X$ 表示矩阵，粗体大写字母 $X, Y$ 表示张量。
假设训练样本表示为第 $n$ 阶张量 $\{\mathbf{X}_i \in \mathbb{R}^{m_1 \times m_2 \times \cdots \times m_n}, i = 1, 2, \ldots, N\}$，其中 $N$ 表示训练样本的总数。我们需要以下符号和定义，类似于[12]-[14], [19], [27]。
**定义 1**：第 $n$ 阶张量 $\mathbf{X} \in \mathbb{R}^{m_1 \times m_2 \times \cdots \times m_n}$ 的 mode-$k$ 展开为矩阵 $X^k \in \mathbb{R}^{m_k \times \prod_{i \neq k} m_i}$，即 $X^k \Leftarrow_k \mathbf{X}$，定义为 $X_{ik,j}^k = \mathbf{X}_{i_1, i_2, \ldots, i_n}$，其中 $j = 1 + \sum_{l \neq k} (i_l - 1) \prod_{o = l + 1, o \neq k}^{n} m_o$，其中 $i_1, i_2, \ldots, i_n$ 表示索引。
关于第 $n$ 阶张量 mode-$k$ 展开的直观表示，读者请参考[19]中的图。
**定义 2**：张量 $\mathbf{X}$ 与矩阵 $U \in \mathbb{R}^{m_k' \times m_k}$ 的 mode-$k$ 乘积定义为 $\mathbf{Y} = \mathbf{X} \times_k U$，其中 $\mathbf{Y}_{i_1, \ldots, i_{k-1}, i, i_{k+1}, \ldots, i_n} = \sum_{j=1}^{m_k} \mathbf{X}_{i_1, \ldots, i_{k-1}, j, i_{k+1}, \ldots, i_n} U_{i, j}$，$j = 1, \ldots, m_k'$。
在不影响一般性的情况下，我们假设训练张量样本是中心化的，并且仍表示为 $\{\mathbf{X}_i \in \mathbb{R}^{m_1 \times m_2 \times \cdots \times m_n}, i = 1, 2, \ldots, N\}$。

#### 2.2. SPCA

PCA可以被表述为一个岭回归优化问题。因此，通过对回归问题施加lasso约束，可以获得SPCA的稀疏投影。也就是说，SPCA考虑以下弹性网正则化问题：
$$(\hat{Q},\hat{P})=\arg\min\sum_{i}\left\|x_{i}-QP^{T}x_{i}\right\|^{2}+\alpha\parallel P\parallel_{F}^{2}+\sum_{j}^{d_{1}}\beta_{j}\left|\mathbf{p}_{1}^{j}\right|$$
(1)
其中约束条件为 $Q^TQ=I_d$。
其中，$x_i$ 是由张量 $\mathcal{X}_i$ 的列/行连接而成的高维向量，$Q$ 和 $P$ 是 $d \times (m_1 \times m_2 \times \cdots \times m_n)$ 的矩阵，$\alpha,\beta_j \geq 0$ 用于惩罚不同主成分向量的载荷。定义在（1）中的问题可以通过交替最小化算法来计算最优的 $\hat{Q}$ 和 $\hat{P}$。然后，稀疏主成分向量为 $\hat{p}_{i}/\left\|\hat{p}_{i}\right\|,(i=1,2,\ldots,d)$，可以用于特征提取。

#### 2.3. MPCA
MPCA的目的是获得一组多线性变换（或投影）$V_i \in \mathbb{R}^{m_i \times d_i}$，$(d_i \leq m_i, i = 1, 2, \ldots, n)$，将原始高阶张量数据映射到一个低阶张量空间：
$$
\mathbf{Y}_i=\mathbf{X}_i\times_1V_1^T\times_2V_2^T\cdots\times_nV_n^T
$$
(2)
MPCA的目标函数是在子空间 $V_i \in \mathbb{R}^{m_i \times d_i}$ 上直接最大化总散射矩阵：
$$
\max\limits_{V_k^TV_k=I_k}\:\mathrm{tr}\big(V_k^TS_T^kV_k\big)=\max\limits_{V_k^TV_k=I_k}\:\mathrm{tr}\big[V_k^T\big(\sum_i^N X_i^k X_i^{kT}\big)V_k\big]
$$
(3)
其中 $S_T^k = \sum_i^N X_i^k X_i^{kT}$，而 $X_i^k$ 是 $\mathbf{X}_i$ 在 mode-$k$ 展开的矩阵：
$$
X_i^k \in \mathbf{X}_i \times_1 V_1^T \times \cdots \times_{k-1} V_{k-1}^T \times_{k+1} V_{k+1}^T \times \cdots \times_n V_n^T.
$$
MPCA的最优投影可以通过SVD或特征分解来获得：
$$
S_T^k V_k = V_k D_k
$$
(4)
其中 $V_k = [\mathbf{v}_k^1, \mathbf{v}_k^2, \ldots, \mathbf{v}_k^{d_k}]$ 是特征向量矩阵，$D_k = \operatorname{diag}(\lambda_k^1, \lambda_k^2, \ldots, \lambda_k^{d_k})$ 是 $S_T^k$ 的特征值矩阵，$\lambda_k^1 \geq \lambda_k^2 \geq \ldots \geq \lambda_k^{d_k}$，$\lambda_k^j$ 是对应于特征向量 $\mathbf{v}_k^j$ 的特征值。对于每个模式，MPCA的最优投影矩阵由对应于前 $d_k$ 个最大特征值的特征向量组成。即，$V_k = [\mathbf{v}_k^1, \mathbf{v}_k^2, \ldots, \mathbf{v}_k^{d_k}]$ 是每个模式的MPCA投影矩阵。

#### 2.4. MSPCA

在本节中，首先介绍MPCA的多线性回归。然后，将其修改为多线性稀疏主成分学习。
##### 2.4.1. 用于MPCA的多线性岭回归
众所周知，PCA可以表示为回归形式。同样地，我们将单线性回归扩展为多线性回归，其目的是通过$L_{2}$范数惩罚最小化张量重构误差。设$B_i \in \mathbb{R}^{m_i \times d_i} (i = 1, 2, \ldots, N)$，并且
$$
\begin{aligned}
J(B_1, B_2, \ldots, B_n) &= \sum_{i} \left\| \mathbf{X}_i - \mathbf{X}_i \times_{1} B_1 B_1^T \times_{2} B_2 B_2^T \ldots \times_{n} B_n B_n^T \right\|_{F}^2 \\
&+ \sum_{j} \alpha_j \left\| B_j \right\|_{F}^2.
\end{aligned}
$$
(5)
据我们所知，对于如此复杂的目标函数不存在闭式解。幸运的是，该优化问题可以转换为一个独立确定$n$个子空间$B_n$的问题，这些子空间使用类似于MPCA的迭代算法来最小化第$n$阶张量的mode-$k$展开的构造误差。因此，在以下部分中，我们仅关注第$n$阶张量的mode-$k$展开。假设$B_1, B_2, \ldots, B_{k-1}, B_{k+1}, \ldots, B_n$是已知的，根据[11]中的定理1，(6)中的最小化问题转换为以下带单一约束的优化问题：
$$
\min_{B_k} \sum_{i} \left\| X_i^k - B_k B_k^T X_i^k \right\|_{F}^2 + \alpha_k \left\| B_k \right\|_{F}^2 \\
\mathrm{s.t.} \quad B_k^T B_k = I_k.
$$
(7)
从(7)中，我们得到
$$
\begin{aligned}
\min_{B_k} &\sum_{i} \left\| X_i^k - B_k B_k^T X_i^k \right\|_{F}^2 + \alpha_k \left\| B_k \right\|_{F}^2 \\
&= \min_{B_k} 2 \mathrm{tr}(S_T^k) - \mathrm{tr}\big[B_k^T(S_T^k + \alpha_k I_k)B_k\big].
\end{aligned}
$$
(8)
最小化(8)等价于
$$
\max_{B_k^T B_k = I_k} \mathrm{tr}\Big[B_k^T (S_T^k + \alpha_k I_k) B_k \Big].
$$
(9)
由于$\alpha_k I_k$不影响特征向量，(4)和(9)具有相同的解，即$V_k$是(9)的最优解。因此，MPCA的特征向量可以通过多线性回归得出。
##### 2.4.2. MSPCA的模型松弛
由于多线性特征提取方法侧重于第$n$阶张量的mode-$k$展开的优化问题，以下我们关注多线性回归问题(7)。设$U_k \in \mathbb{R}^{m_k \times d_k}$，为了获得稀疏主成分向量的稀疏回归模型，我们将(7)松弛为一个新的回归问题：
$$
\begin{aligned}
\min_{U_k, B_k} &\sum_{i} \parallel X_i^k - B_k U_k^T X_i^k \parallel_{F}^2 + \alpha_k \parallel U_k \parallel_{F}^2 \\
&\mathrm{s.t.} \quad B_k^T B_k = I_k.
\end{aligned}
$$
(10)

**Plus：松弛求解**

翻译如下：
证明：从(10)式，我们有 
$$
\begin{aligned}
&\sum_{i}\|X_{i}^{k}-B_{k}U_{k}^{T}X_{i}^{k}\|_{F}^{2}+\alpha_{k}\|U_{k}\|_{F}^{2} \\
&=\mathrm{tr}\biggl(\sum_{i}X_{i}^{k}X_{i}^{k^{T}}\biggr)-2\mathrm{tr}\biggl[B_{k}^{T}\biggl(\sum_{i}X_{i}^{k}X_{i}^{k^{T}}\biggr)U_{k}\biggr] \\
&+\mathrm{tr}\bigg[U_k^T\bigg(\sum_iX_i^kX_i^{k^T}+\alpha I_k\bigg)U_k\bigg] \\
&=\mathrm{tr}\big(S_{T}^{k}\big)-2\mathrm{tr}\big(B_{k}^{T}S_{T}^{k}U_{k}\big)+\mathrm{tr}\big[U_{k}^{T}\big(S_{T}^{k}+\alpha I_{k}\big)U_{k}\big].
\end{aligned}
$$
对于固定的 $B_k$，使用拉格朗日乘子法，我们知道上述量在
$$\hat{U}_k=\left(S_T^k+\alpha I_k\right)^{-1}S_T^kB_k$$
时最小。
(A2)
将(A2)代入(A1)，我们有
$$\begin{aligned}\sum_{i}\left\|X_{i}^{k}-B_{k}U_{k}^{T}X_{i}^{k}\right\|_{F}^{2}+\alpha_{k}\left\|U_{k}\right\|_{F}^{2}\\&=\mathrm{tr}(S_{T}^{k})-\mathrm{tr}(B_{k}^{T}S_{T}^{k}(S_{T}^{k}+\alpha I_{k})^{-1}S_{T}^{k}B_{k}).\end{aligned}$$
(A3)
因此，最小化(A3)等价于以下最大化问题：
(A4)
$$\arg\max_{B_{k}}\mathrm{tr}(B_{k}^{T}S_{T}^{k}(S_{T}^{k}+\alpha I_{k})^{-1}S_{T}^{k}B_{k})\\\mathrm{s.t.}\quad B_{k}^{T}B_{k}=I_{k}.$$
由于 $V_{k}$ 是(9)的解，因此它也是(7)的解。记 $S_T^k$ 的奇异值分解为 $\widehat{V}_k\widehat{D}_k\widehat{V}_k^T$，我们有
$$S_T^k(S_T^k+\alpha I_k)^{-1}S_T^k=\hat{V}_k\frac{\hat{D}_k^2}{\hat{D}_k+\alpha I_k}\hat{V}_k^T.$$
(A5)
从(A5)可以很容易看出 $B_{k}=V_{k}$ 是(A4)的解。因此，从(A2)我们得到
$$\hat{U}_k=(S_T^k+\alpha I_k)^{-1}S_T^kB_k=V_k\frac{\hat{D}_k(1:d_k,1:d_k)}{\hat{D}_k(1:d_k,1:d_k)+\alpha I_{d_k}}$$
(A6)
其中 $\widehat{D}_{k}(1:d_{k},1:d_{k})$ 表示 $\widehat{D}_{k}$ 的子矩阵。因此，
$$\mathbf{u}_{k}^{j}\propto\mathbf{v}_{k}^{j}\:(1\leq j\leq d_{k}).$$

验毕

以下定理揭示了(7)（或(6)）和(10)之间的紧密关系。

**定理1**：对于任何给定的$a_k > 0$，(7)和(10)中的优化问题对于变量$k$具有相同的解。设$(\hat{B}_k, \hat{U}_k)$为(10)的最优解，其中$k = [\mathbf{u}_k^1, \mathbf{u}_k^2, \ldots, \mathbf{u}_k^{d_k}]$，证明见补充附录。
定理1揭示了多线性回归问题和MPCA目标函数在任意给定mode $k$上的紧密关系。由于(6)的最优解由(7)给出，而(10)也具有相同的解，因此是一种使用$L_{1}$范数惩罚计算稀疏向量的可行方法。因此，为了获得多线性稀疏主成分向量，将lasso惩罚施加在MPCA回归准则的mode-$k$展开的回归表示上。最终，MSPCA准则定义如下：
$$
\min_{B_k, U_k} \sum_{i} \left\| X_i^k - B_k U_k^T X_i^k \right\|_{F}^2 + \alpha_k \left\| U_k \right\|_{F}^2 + \sum_{j}^{d_k} \beta_{k,j} \left| \mathbf{u}_k^j \right| (\forall k)
$$
(11)
$$
\mathrm{s.t.} \quad B_k^T B_k = I_k
$$
其中$\beta_{k,j} \geq 0$用于惩罚不同主成分向量的载荷。我们通常假设$\beta_{k,j} > 0$，这会导致多线性稀疏主成分向量。

##### 2.4.3. MSPCA的解法

### C. MSPCA 的解法
本节介绍了如何计算MSPCA的稀疏解。根据(11)，我们有
$$
\begin{aligned}
&\sum_{i} \|X_{i}^{k} - B_{k}U_{k}^{T}X_{i}^{k}\|_{F}^{2} + \alpha \|U_{k}\|_{F}^{2} + \sum_{j}^{d_{k}} \beta_{k,j} |\mathbf{u}_{k}^{j}| \\
&= \mathrm{tr} \bigg( \sum_{i} X_{i}^{k} X_{i}^{k^{T}} \bigg) + \sum_{j} \bigg( \mathbf{u}_{k}^{jT} \bigg[ \bigg( \sum_{i} X_{i}^{k} X_{i}^{k^{T}} \bigg) + \alpha \bigg] \mathbf{u}_{k}^{j} \\
&- 2 \mathbf{b}_{k}^{jT} \bigg( \sum_{i} X_{i}^{k} X_{i}^{k^{T}} \bigg) \mathbf{u}_{k}^{j} + \beta_{k,j} \big| \mathbf{u}_{k}^{j} \big| \bigg) \\
&= \mathrm{tr}(S_{T}^{k}) + \sum_{j} \left( \mathbf{u}_{k}^{jT} (S_{T}^{k} + \alpha) \mathbf{u}_{k}^{j} - 2 \mathbf{b}_{k}^{jT} S_{T}^{k} \mathbf{u}_{k}^{j} + \beta_{k,j} \big| \mathbf{u}_{k}^{j} \big| \right).
\end{aligned}
$$
(12)
可以观察到，如果 $B_k$ 已知，那么 (12) 的最优稀疏解正是以下 $m_k^{\prime}$ 个独立的朴素弹性网问题的解：
$$
\mathbf{u}_{k}^{jT} (S_{T}^{k} + \alpha) \mathbf{u}_{k}^{j} - 2 \mathbf{b}_{k}^{jT} S_{T}^{k} \mathbf{u}_{k}^{j} + \beta_{k,j} |\mathbf{u}_{k}^{j}| \quad j=1, \ldots, m_{k}^{\prime}.
$$
另一方面，我们也有
$$
\begin{aligned}
\sum_{i} \|X_{i}^{k} - B_{k}U_{k}^{T}X_{i}^{k}\|_{F}^{2} + \alpha \|U_{k}\|_{F}^{2} + \sum_{j}^{d_{k}} \beta_{k,j} |\mathbf{u}_{k}^{j}| \\
= \mathrm{tr} \bigg( \sum_{i} X_{i}^{k} X_{i}^{k^{T}} \bigg) - 2 \mathrm{tr} \bigg[ B_{k}^{T} \bigg( \sum_{i} X_{i}^{k} X_{i}^{k^{T}} \bigg) U_{k} \bigg] \\
+ \mathrm{tr} \bigg[ U_{k}^{T} \bigg( \sum_{i} X_{i}^{k} X_{i}^{k^{T}} + \alpha I_{k} \bigg) U_{k} \bigg] + \sum_{i} \beta_{k,j} |\mathbf{u}_{k}^{j}|
\end{aligned}
$$
(14)
当 $U_k$ 已知且固定时，(14) 的第一、三、四项是常数，因此可以忽略。最后，最小化(14)的问题变成了以下最大化问题：
$$
\max \: \mathrm{tr} \bigg[ B_k^T \bigg( \sum_i X_i^k X_i^{k^T} \bigg) U_k \bigg] \\
\mathrm{s.t.} \quad B_{k}^{T} B_{k} = I_{k}.
$$
(15)
根据[26]中的定理4，给定 $U_k$ 的上述最大化问题的最优解为
$$
B_k^* = \bar{U} \bar{V}^T
$$
(16)
其中 $\bar{U}$ 和 $\bar{V}$ 是 $ \sum_i X_i^k X_i^{k^T}  U_k$ 的 SVD 分解，即 $(\sum_{i} X_{i}^{k} X_{i}^{k^{T}}) U_{k} = \bar{U} \bar{D} \bar{V}$。MSPCA算法的详细信息见补充附录。


### 3. 摘译

#### 3.1 摘要

摘要—本文提出了用于从张量数据中进行特征提取的多线性稀疏主成分分析（MSPCA）。MSPCA可以看作是对经典主成分分析（PCA）、稀疏主成分分析（SPCA）以及最近提出的多线性主成分分析（MPCA）的进一步扩展。MSPCA的关键操作是将MPCA重写为多线性回归形式，并将其松弛为稀疏回归。与最近提出的MPCA不同，MSPCA继承了SPCA的稀疏性，并迭代地学习一系列稀疏投影，捕获了张量数据大部分的变化。稀疏投影中的每个非零元素都是使用弹性网从最重要的变量/因子中选择的。对Yale、人脸识别技术人脸数据库和COIL-20对象数据库进行的广泛实验，这些数据库将对象图像编码为二阶张量，并使用Weizmann动作数据库作为三阶张量，证明了所提出的MSPCA算法有望胜过现有的基于PCA的子空间学习算法。